In [1]:
import pandas as pd
import re
import numpy as np
import feather
import string
import os
import seaborn as sns

dataset = input()
path = os.getcwd() + '/Datasets/'+dataset+'/'

banks


In [2]:
#df = pd.read_feather('Datasets/#fintech.csv')
df = pd.read_feather(path+dataset+'18_21.ftr')
df.shape

(1537286, 15)

In [3]:
# Append 15-18 tweets
df_old = pd.read_feather(path+dataset+'15_18.ftr')
df = df.append(df_old, ignore_index = True)
# Append 09-15 tweets
df_old = pd.read_feather(path+dataset+'09_15.ftr')
df = df.append(df_old, ignore_index = True)

df.shape

(4502569, 15)

# Data Cleaning

## 1) Filter and Clean the Dataframe

In [4]:
#Next, mostly to get rid of bots and cryptocurrency warriors, 
#we are going to remove users that have less than 100 followers:
df['sum_pubmetrics'] = df['public_metrics.retweet_count'] + df['public_metrics.like_count']
df = df[df['public_metrics.followers_count']>=100]
df.shape

(4018801, 16)

In [5]:
#Reset the index to start from 0 and drop the previous index columns
df = df.reset_index().drop(columns=['index'])

In [6]:
# For banking create a dummy variable that shows whether the tweet contains the word/hashtag fintech
df['fintech'] = df.text.str.contains('fintech', case = False)

In [7]:
df.to_feather(path+dataset+'_full.ftr')
df.to_csv(path+dataset+'_full.csv')

### 1.1) Clean the tweets

In [8]:
def clean_text_round1(text):
    """Initial cleaning I make to the dataset"""
    text = re.sub(r'@[A-Za-z0-9]+','',text) #Removes @mentions
    text = re.sub(r'#','',text) #Removes the '#' symbol
    text = re.sub(r'&amp;','',text) #Removes &amp
    text = re.sub(r'https?:\/\/\S+','',text) #Remove the hyper link
    text = re.sub(r'\n','',text) #remove the new line symbol
    text = re.sub(r'\t','',text) #remove the tab symbol
    text = re.sub(r'-',' ',text) #removes the dash and replaces it with space
    #Removes the punctuation:
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text) #remove words containing numbers
    text = re.sub(r"[^a-zA-Z0-9]+", " ", text) #Removes special characters except space
    text = re.sub(' +', ' ', text).strip() #Removes trailing white spaces
    text = text.lower() # lowers the text
    text = re.sub(r"(.)\1{2,}",r"\1", text) # Replace triple+ characters with single char
    
    return text

In [9]:
df['text'] = df['text'].apply(clean_text_round1)

## 2) Organize the Data

### 2.1) Corpus

For corpus I will group by years and make separate corpuses/columns for different years.

In [ ]:
#df = pd.read_feather(path+dataset+'_clean.ftr')

In [11]:
# Set the index to be the date the tweet was created
df = df.set_index('tweet_created_at')
# Group the tweets by year and merge the tweets by separating them by space:
df = df.text.groupby(pd.Grouper(freq='Y')).apply(' '.join)
# Rename the index to year ending in:
df = df.rename_axis('year_ending_in')
# Save it as a corpus:
#df.to_frame().reset_index().to_feather('Datasets/#'+dataset+'_corpus.ftr')
df.to_pickle(path+dataset+'_corpus.pkl')

### 2.2) Document-Term Matrix

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

In [13]:
cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(df)

In [14]:
# Create a pandas dataframe: columns are the words appearing in the tweets,
# rows are showing the frequency of these words
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
# Set the index to be dates:
data_dtm.index = df.index

print('The shape of the document-term matrix is:')
print(data_dtm.shape)

The shape of the document-term matrix is:
(13, 789413)


In [15]:
# Save the dtm:
data_dtm.transpose().to_pickle(path+dataset+'_dtm.pkl')